#### Code to create Csv file of dataset with less accurate model

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import pickle

In [2]:
# Loading SVM model
list_unpickle = open("model", 'rb')
clf = pickle.load(list_unpickle)

In [3]:
def predictSingleImage(img): 
    debug = not True
    kernel = np.ones((2,2),np.uint8)
    bordersize = 10
    kernel2 = np.ones((12,12),np.float32)/225
    # Basic Processing
    resize = cv2.resize(img, (400,200))
    border = cv2.copyMakeBorder(
    resize,
    top=bordersize,
    bottom=bordersize,
    left=bordersize,
    right=bordersize,
    borderType=cv2.BORDER_CONSTANT,
    value=[255, 255,255]
    )
    gray = cv2.cvtColor(border,cv2.COLOR_BGR2GRAY)
    smoothed = cv2.filter2D(gray,-1,kernel2)
    retval,thresh = cv2.threshold(smoothed, 30, 255, cv2.THRESH_BINARY)
    erode = cv2.erode(thresh,kernel,iterations = 2)
    if debug:
        cv2.imshow("Gray", gray)
        cv2.waitKey(0)
        cv2.imshow("Smooth", smoothed)
        cv2.waitKey(0)
        cv2.imshow("Thresholding", thresh)
        cv2.waitKey(0)
        cv2.imshow("Dilate", dialate)
        cv2.waitKey(0)
    final = erode
    # Find Contours
    contourImg = np.copy(final)
    _, contours, hie = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    hie = hie[0]
    goodContours = []
    ind = 0
    for c in contours:
        testimg = np.copy(contourImg)
        rect = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        x,y,w,h = rect
        cv2.rectangle(testimg,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.waitKey(1)
        parent = hie[ind][3]
        ind+=1
        if(parent == 0):
            goodContours.append(c)
    cv2.destroyAllWindows()
    rects = []
    for c in goodContours:
        rect = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        x,y,w,h = rect
        if(w > 150):
            x2 = x + int(w/3)
            x3 = x2 + int(w/3)
            rects.append((x2,y,int(w/3),h))
            rects.append((x3,y,int(w/3),h))
            rects.append((x,y,int(w/3),h))
            cv2.rectangle(contourImg,(x2,y),(x2+int(w/3),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x3,y),(x3+int(w/3),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x,y),(x+int(w/3),y+h),(0,255,0),2)
        elif (w > 65):
            x2 = x + int(w/2)
            rects.append((x2,y,int(w/2),h))
            rects.append((x,y,int(w/2),h))
            cv2.rectangle(contourImg,(x2,y),(x2+int(w/2),y+h),(0,255,0),2)
            cv2.rectangle(contourImg,(x,y),(x+int(w/2),y+h),(0,255,0),2)
        else:
            rects.append((x,y,w,h))
            cv2.rectangle(contourImg,(x,y),(x+w,y+h),(0,255,0),2)
    rects = sorted(rects)
    if (len(rects) != 5):
        return
    ans = ''
    i = 0
    for (x,y,w,h) in rects:
        roi = contourImg[y:y+h,x:x+w]
        display = cv2.resize(roi, (120,120))
        roi = cv2.resize(roi, (20,20))
#         print(roi.shape) 
        pred = clf.predict(np.reshape(roi,(1,400)))
        ans+=pred[0]
#         print('p: ',pred)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    cv2.destroyAllWindows()
    return ans

In [4]:
path = os.getcwd()
names = os.listdir(os.path.join(path,'digitdata'))
names.sort()
os.path.join('digitdata',names[0])
prediction = []
for imgfile in names:
    prediction.append(predictSingleImage(cv2.imread(os.path.join('digitdata',imgfile))))

print('Prediction ',len(prediction))
print('Images ',len(names))
df = pd.DataFrame({'image_names': names, 'value': prediction})
df.to_csv('CAP01.csv', index=False, encoding='utf-8')

Prediction  1092
Images  1092
